## Prepare training data

In [2]:
import torch
import numpy as np

# 定义序列参数
batch_size = 400
sequence_length = 100
input_size = 1

clean_signal = torch.sin(
    torch.linspace(0, 2 * np.pi * batch_size, batch_size * sequence_length)
    ).unsqueeze(-1)
print("clean_signal shape:", clean_signal.size())
clean_batches = clean_signal.view(batch_size, sequence_length, input_size)
print("shape of clean_batches:", clean_batches.size())

# 在正弦波上叠加噪声
noise = torch.randn_like(clean_signal) * 0.2  # 调整噪声强度
noisy_signal = clean_signal + noise
print("noisy_signal shape:", noisy_signal.size())
noisy_batches = noisy_signal.view(batch_size, sequence_length, input_size)
print("shape of noisy_batches:", noisy_batches.size())

clean_signal shape: torch.Size([40000, 1])
shape of clean_batches: torch.Size([400, 100, 1])
noisy_signal shape: torch.Size([40000, 1])
shape of noisy_batches: torch.Size([400, 100, 1])


In [3]:
import pandas as pd
from src.utils import plotters

# Plot noisy_signal 和 clean_signal
noisy_signal_flat = noisy_signal.view(-1).numpy()  # 展平为一维数组，形状 (40000,)
clean_signal_flat = clean_signal.view(-1).numpy()  # 展平为一维数组，形状 (40000,)
plotters.plot_traces(
    pd.DataFrame({
        "Noisy Signal": noisy_signal_flat,
        "Clean Signal": clean_signal_flat
    }),
    width=800,
    height=400,
    mode="lines"
)

# plot batch 1 signal
noisy_batches_flat = noisy_batches[0].view(-1).numpy()  # 展平为一维数组，形状 (40000,)
clean_batches_flat = clean_batches[0].view(-1).numpy()  # 展平为一维数组，形状 (40000,)
plotters.plot_traces(
    title="batch 1",
    data = pd.DataFrame({
                "Noisy Signal": noisy_batches_flat,
                "Clean Signal": clean_batches_flat
            }),
    width=800,
    height=400,
    mode="lines"
)

## Create dataloader

In [4]:
from torch.utils.data import Dataset, DataLoader

# 定义自定义数据集
class TimeSeriesDataset(Dataset):
    def __init__(self, noisy_batches, clean_batches):
        self.noisy_batches = noisy_batches
        self.clean_batches = clean_batches

    def __len__(self):
        return len(self.noisy_batches)

    def __getitem__(self, idx):
        return self.noisy_batches[idx], self.clean_batches[idx]

# 定义数据集和数据加载器
dataset = TimeSeriesDataset(noisy_batches, clean_batches)
dataloader = DataLoader(dataset, batch_size=40, shuffle=False)

print(f"Number of batches in dataloader: {len(dataloader)}")

Number of batches in dataloader: 10


## Define model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义 LSTM 降噪模型
class LSTMDenoiser(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 初始化模型
model = LSTMDenoiser(input_size=1, hidden_size=64, num_layers=2)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## Train model

In [5]:
import os

# 用于保存训练损失
epoch_losses  = []

num_epochs = 20  # 训练轮数

# 初始化变量以跟踪最佳模型
best_loss = float('inf')
best_model_path = "models/best_model.pth"
last_model_path = "models/last_model.pth"

# 检查是否存在之前的模型
if os.path.exists(best_model_path):
    print("Loading checkpoint...")
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_loss = checkpoint['loss']
    best_epoch = checkpoint['epoch'] + 1
    print(f"Resuming training from epoch {start_epoch} with best loss {best_loss:.8f}")
else:
    print("No checkpoint found, starting fresh training.")

# 模型训练
for epoch in range(num_epochs):
    epoch_loss = 0
    for i, (noisy_batch, clean_batch) in enumerate(dataloader):
        # 将数据移动到 GPU（如果可用）
        noisy_batch = noisy_batch.float()
        clean_batch = clean_batch.float()

        # 前向传播
        outputs = model(noisy_batch)
        loss = criterion(outputs, clean_batch)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 累加 batch 的损失
        epoch_loss += loss.item()

    # 记录每个 epoch 的平均损失
    avg_loss = epoch_loss / len(dataloader)
    epoch_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.8f}")

    # 检查是否为最佳模型
    if avg_loss < best_loss: 
        best_epoch=epoch+1
        best_loss = avg_loss
        # torch.save(model.state_dict(), best_model_path)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_loss,
            },
            best_model_path
        )
        print(f"New best model saved: {best_model_path} with loss: {best_loss:.8f}")

    # 保存当前 epoch 的模型
    # torch.save(model.state_dict(), last_model_path)
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
        },
        last_model_path
    )
    print(f"Last model saved at epoch {epoch+1}")

print(f"Training complete! Best model was from epoch {best_epoch} with loss: "
      f"{best_loss:.8f}")

Loading checkpoint...
Resuming training from epoch 19 with best loss 0.00058562
Epoch [1/100], Average Loss: 0.00090876
Last model saved at epoch 1


C:\Users\hrsxz\AppData\Local\Temp\ipykernel_3432\4028586620.py:16: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Epoch [2/100], Average Loss: 0.00094747
Last model saved at epoch 2
Epoch [3/100], Average Loss: 0.00050321
New best model saved: models/best_model.pth with loss: 0.00050321
Last model saved at epoch 3
Epoch [4/100], Average Loss: 0.00065264
Last model saved at epoch 4
Epoch [5/100], Average Loss: 0.00056497
Last model saved at epoch 5
Epoch [6/100], Average Loss: 0.00059556
Last model saved at epoch 6
Epoch [7/100], Average Loss: 0.00059340
Last model saved at epoch 7
Epoch [8/100], Average Loss: 0.00058624
Last model saved at epoch 8
Epoch [9/100], Average Loss: 0.00060004
Last model saved at epoch 9
Epoch [10/100], Average Loss: 0.00058870
Last model saved at epoch 10
Epoch [11/100], Average Loss: 0.00059056
Last model saved at epoch 11
Epoch [12/100], Average Loss: 0.00058648
Last model saved at epoch 12
Epoch [13/100], Average Loss: 0.00058120
Last model saved at epoch 13
Epoch [14/100], Average Loss: 0.00057852
Last model saved at epoch 14
Epoch [15/100], Average Loss: 0.00057381

## Plot results

In [6]:
import plotly.graph_objects as go

# 绘制训练损失曲线
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=epoch_losses, mode='lines', name='Training Loss'))
fig2.update_layout(title='Training Loss Curve', xaxis_title='Epoch', yaxis_title='Loss')
fig2.show()

## Inference data

In [26]:
# 加载模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 假设你有一个定义好的模型类
model = LSTMDenoiser(input_size=1, hidden_size=64, num_layers=2)
model.to(device)

# 加载已训练的模型权重
checkpoint = torch.load("models/best_model.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # 设置模型为评估模式

# 预测降噪结果
with torch.no_grad():
    denoised_signal = model(noisy_batches[0])  # 添加 batch 维度 (1, sequence_length, input_size)
    denoised_signal = denoised_signal.squeeze(0)  # 移除 batch 维度，形状为 (sequence_length, input_size)

# 绘制降噪结果与干净信号的对比
fig3 = go.Figure()

# 绘制第一个 batch 的噪声信号、干净信号和去噪信号
fig3.add_trace(go.Scatter(y=noisy_batches[0].squeeze().cpu().numpy(), mode='lines', name='Noisy Signal'))  # 噪声信号
fig3.add_trace(go.Scatter(y=clean_batches[0].squeeze().cpu().numpy(), mode='lines', name='Clean Signal'))  # 干净信号
fig3.add_trace(go.Scatter(y=denoised_signal.squeeze().cpu().numpy(), mode='lines', name='Denoised Signal'))  # 去噪信号

fig3.update_layout(
    title='Denoised vs Clean Signal',
    xaxis_title='Time Step',
    yaxis_title='Signal Value',
    legend_title='Signal Type'
)

fig3.show()


C:\Users\hrsxz\AppData\Local\Temp\ipykernel_3432\3381845517.py:9: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [34]:
import numpy as np
import torch
import plotly.graph_objects as go

# 模拟信号
sequence_length = 100

# linspace(start, stop, num=50)
clean_signal = np.sin(np.linspace(0, 2 * np.pi , sequence_length)).astype(np.float32)  # 模拟正弦波干净信号
print("clean_signal shape:", clean_signal.shape)
noise = np.random.normal(0, 0.2, sequence_length).astype(np.float32)  # 添加噪声
noisy_signal = clean_signal + noise  # 噪声信号
print("noisy_signal shape:", noisy_signal.shape)

# 转换为 PyTorch 张量
noisy_signal_tensor = torch.from_numpy(noisy_signal).unsqueeze(1).unsqueeze(0)  # (1, sequence_length, 1)
print("noisy_signal_tensor shape:", noisy_signal_tensor.shape)

# 加载模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 假设你有一个定义好的模型类
model = LSTMDenoiser(input_size=1, hidden_size=64, num_layers=2)
model.to(device)

# 加载已训练的模型权重
checkpoint = torch.load("models/best_model.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # 设置模型为评估模式

# 预测降噪结果
with torch.no_grad():
    denoised_signal = model(noisy_signal_tensor)  # 模型输出 (batch_size, sequence_length, input_size)
    print("denoised_signal shape:", denoised_signal.shape)
    
# 绘制对比图
fig3 = go.Figure()

fig3.add_trace(go.Scatter(y=noisy_signal, mode='lines', name='Noisy Signal'))  # 噪声信号
fig3.add_trace(go.Scatter(y=clean_signal, mode='lines', name='Clean Signal'))  # 干净信号
fig3.add_trace(go.Scatter(y=denoised_signal.squeeze().numpy(), mode='lines', name='Denoised Signal'))  # 去噪信号

# 更新布局
fig3.update_layout(
    title='Denoised vs Clean Signal',
    xaxis_title='Time Step',
    yaxis_title='Signal Value',
    legend_title='Signal Type'
)

# 显示图表
fig3.show()

clean_signal shape: (100,)
noisy_signal shape: (100,)
noisy_signal_tensor shape: torch.Size([1, 100, 1])
denoised_signal shape: torch.Size([1, 100, 1])


C:\Users\hrsxz\AppData\Local\Temp\ipykernel_3432\1829333178.py:27: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



## Export onnx format model

In [77]:
# 训练完成后
torch.save(model.state_dict(), "trained_lstm_denoiser.pth")
print("Model training completed and weights saved.")

# 加载训练后的模型权重（可选）
model.load_state_dict(torch.load("trained_lstm_denoiser.pth"))

# 导出为 ONNX 格式
onnx_path = "./models/lstm_denoiser.onnx"
input_data = torch.randn(batch_size, sequence_length, input_size)  # Example input data
torch.onnx.export(
    model,
    input_data,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size", 1: "sequence_length"},
                  "output": {0: "batch_size", 1: "sequence_length"}},
    opset_version=11,
)
print(f"Model exported to {onnx_path}")

Model training completed and weights saved.


C:\Users\hrsxz\AppData\Local\Temp\ipykernel_22532\3255348422.py:6: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Model exported to ./models/lstm_denoiser.onnx


d:\Foester\initial preparation\noise_reduction\.venv\Lib\site-packages\torch\onnx\symbolic_opset9.py:4279: UserWarning:

Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 



## Validate onnx format model

In [78]:
# 创建干净的正弦波信号
time = np.linspace(0, 2 * np.pi, sequence_length)  # 时间步
clean_signal = np.sin(time)  # 单个正弦波信号
clean_signal = np.tile(clean_signal, (batch_size, 1)).reshape(batch_size, sequence_length, input_size)  # 扩展到批次

# 添加随机噪声
noise = np.random.normal(0, 0.2, clean_signal.shape)  # 噪声强度为 0.2
noisy_signal = clean_signal + noise  # 带噪正弦波

# 转换为 float32 类型
input_data = noisy_signal.astype(np.float32)
clean_signal = clean_signal.astype(np.float32)

In [79]:
# 验证onnx格式模型
import onnxruntime as ort
import numpy as np

# 加载 ONNX 模型
session = ort.InferenceSession("models/lstm_denoiser.onnx")

# 推理
outputs = session.run(["output"], {"input": input_data})
print("Output shape:", np.array(outputs).shape)

# 转换为 NumPy 数组
denoised_output = np.array(outputs[0])  # 模型输出，形状 (32, 100, 1)
print("Output shape:", denoised_output.shape)  # 确保形状是 (32, 100, 1)

Output shape: (1, 400, 1000, 1)
Output shape: (400, 1000, 1)


In [80]:
# 绘制第一个样本的输入信号和模型输出
fig = go.Figure()

# 第一个样本的输入信号（展平）
fig.add_trace(go.Scatter(
    y=input_data[0].squeeze(),
    mode='lines',
    name='Noisy Signal (Input)'
))

# 第一个样本的模型输出（展平）
fig.add_trace(go.Scatter(
    y=denoised_output[0].squeeze(),
    mode='lines',
    name='Denoised Signal (Output)'
))


# 第一个样本的模型输出（展平）
fig.add_trace(go.Scatter(
    y=clean_signal[0].squeeze(),
    mode='lines',
    name='clean_signal (label)'
))


# 设置图表标题和轴标签
fig.update_layout(
    title="Noisy vs Denoised Signal (ONNX Model)",
    xaxis_title="Time Step",
    yaxis_title="Signal Value"
)

# 显示图表
fig.show()


## Conclusion

你的模型在训练过程中对输入信号的分布进行了学习，导致它对特定的信号特性（如周期和采样点数）产生了依赖性。这是因为模型的泛化能力通常受到训练数据分布的限制。

具体分析如下：

---

### **1. 训练数据的分布限制**
在训练中，`clean_signal` 的生成方式是固定的：
```python
torch.sin(torch.linspace(0, 2 * np.pi * batch_size, batch_size * sequence_length))
```
- **周期**：训练数据的周期为 \( 2\pi \)，且总共有 \( \text{batch_size} \) 个完整周期。
- **采样点数**：每个周期被均匀划分为 `sequence_length` 个点。

当模型训练完成后，它习惯了这个特定的分布：
- **固定的周期特性**：模型只见过周期为 \( 2\pi \) 的正弦波。
- **固定的采样密度**：模型只见过每个周期均匀划分为 `sequence_length` 个点的输入信号。

因此，在推理阶段，如果 `clean_signal` 的周期或采样点数发生变化，模型会因为没有见过这样的信号分布而预测不准确。

---

### **2. 推理信号分布与训练信号分布不匹配**
在推理中，你的信号是通过以下方式生成的：
```python
np.sin(np.linspace(0, 2 * np.pi, sequence_length)).astype(np.float32)
```
如果 `sequence_length` 或 `2 * np.pi` 的值变化了，信号的以下特性也会随之变化：
1. **周期变化**：
   - 如果信号的总范围不是整数倍的 \( 2\pi \)，正弦波的周期特性将与训练时的分布不一致。
   
2. **采样点数变化**：
   - 如果 `sequence_length` 增加或减少，采样密度也会改变，这会影响模型对信号的理解。

这些变化导致了训练分布和推理分布的不一致，模型因此无法很好地泛化到新的输入信号。

---

### **3. 模型对输入特征的依赖性**
深度学习模型对输入特征具有很强的依赖性。如果训练过程中输入特征的变化范围有限（如固定的周期和采样点数），模型会“记住”这种特性，而不是学习到泛化的信号降噪规律。

例如：
- 如果训练时只见过周期为 \( 2\pi \) 的正弦波，模型可能会依赖信号的这种周期性特性来进行降噪。
- 如果训练时采样点数固定，模型可能会将信号的采样密度作为特征的一部分，而非完全学习信号的本质。

---

### **4. 如何改进模型的泛化能力**

为了让模型能够处理不同的周期和采样点数，可以尝试以下方法：

#### **(1) 在训练中增加多样性**
在训练数据中引入多样化的信号分布，例如：
- 使用不同的周期范围：
  ```python
  period = np.random.uniform(1, 10)  # 随机选择周期
  clean_signal = torch.sin(torch.linspace(0, 2 * np.pi * period, batch_size * sequence_length)).unsqueeze(-1)
  ```
- 使用不同的采样点数：
  ```python
  sequence_length = np.random.choice([50, 100, 200])  # 随机选择采样点数
  ```

#### **(2) 数据归一化**
对输入信号进行归一化处理，使其周期和采样点数变化时依然可以被模型识别：
```python
noisy_signal = (noisy_signal - noisy_signal.mean()) / noisy_signal.std()
```

#### **(3) 数据增强**
在训练中对信号进行随机缩放、平移等增强操作：
```python
scale_factor = np.random.uniform(0.5, 2.0)  # 随机缩放因子
clean_signal = torch.sin(torch.linspace(0, 2 * np.pi * batch_size, batch_size * sequence_length) * scale_factor)
```

#### **(4) 增加模型的表达能力**
使用更复杂的模型架构，如多层 LSTM 或 Transformer，以捕捉更多的信号特征。